# 2.5 - Ingeniería de Características

* En este Notebook vamos a ver algunos de los conceptos más importantes de lo que se conoce como Ingeniería de Características, cuyo problema consiste en transformar los datos en crudo (raw data) a unos datos limpios, completos y preparados para ser usados con alguna finalidad; como por ejemplo, ser usados por algún algoritmo de aprendizaje:
<span></span><br>
    1. [Tipos de Variables](#M1)
<span></span><br>
    2. [Codificación de Variables Discretas](#M2)
    <span></span><br>
        2.1. [Codificación Binaria](#M21)
    <span></span><br>
        2.2. [Codificación One Hot Encode](#M22)
    <span></span><br>
        2.3. [Label Encoder](#M23)
<span></span><br>
    3. [Discretización](#M3)
<span></span><br>
    4. [Normalización](#M4)
<span></span><br>
    5. [Estandarización](#M5)
<span></span><br>
    6. [Imputación de valores nulos](#M6)


<hr>


# <a name="M1">1. Tipos de Variables</a>

* Los típos de variables estadísticas que nos podemos encontrar son las siguientes:


<img src="../imgs/2_05_01_ic.png" style="width: 800px;"/>


<hr>

# <a name="M2">2. Codificación de Variables Discretas</a>

* ***Los algorítmos de aprendizaje no tienen ningún problema a la hora de trabajar con variables cuantitativas ya que estas siendo discretas (por ejemplo: 1, 2, 3, ... N) o continuas ($-\infty$ a $\infty$)*** pueden trabajar con ellas.


* Sin embargo las ***variables Cualitativas o Categorias (por ejemplo {Madrid, Barcelona, Valencia}, {Primera, Segunda, Tercera}) son Strings y los algoritmos de aprendizaje no pueden trabajar con ellas por tanto hay que transformarlas a números***.


* Distinguimos dos tipos de variables Cualitativas o Categóricas que son:
<span></span><br><br>
    + ***Nominales***: No admiten orden, por ejemplo {Hombre, Mujer} o {Madrid, Barcelona, Valencia}
<span></span><br><br>
    + ***Ordinales***: Admiten un orden entre ellas como por ejemplo {Primera, Segunda, Tercera}


* Dependiendo del tipo de variables Cualitativas o Categóricas debemos de realizar una transformación un otra.
        
        
* ***Variables Nominales***: Debemos de codificar la salida de tal manera que podamos distiguir los tipos. Esto lo podemos hacer con una codificación binária en caso de tener 2 valores o con un "One Hot Encode" en caso de tener más valores. Veamos unos ejemplos:
 
    * ***Codificación Binaria***:
    
|Genero|Codificación|
|---|---|
|Masculino|0|
|Femenino|1|

   * ***Codificación One Hot Encode***:
   
|Ciudad|Madrid|Barcelona|Valencia|
|---|---|---|---|
|Madrid|1|0|0|
|Barcelona|0|1|0|
|Valencia|0|0|1|


* ***Variables Ordinales***: Al ser unas variables que admiten orden, podemos transformar los String a variables numéricas, poniendo el orden que queramos definir. Esta transformación se conoce como "Label Encode". Veamos un ejemplo:

    * ***Codificación Label Encode***:
    
|Categoria|Cat_Ordenada_1|Cat_Ordenada_2|
|---|---|---|
|Primera|1|3|
|Segunda|2|2|
|Tercera|3|1|


* Veamos a continuación como hacer este tipo de Transformaciones:



<hr>

### Carga de Datos

* En primer lugar vamos a definir un DataFrame de ejemplo con el que poder ver estas transformaciones.


* El siguiente DataFrame tiene las siguientes variables que las clasificaremos como:

    + ***Edad***: Variable Continua
    + ***Genero***: Variable Categorica Nominal (2 valores)
    + ***Ciudad_embarque***:Variable Categorica Nominal (>2 valores)
    + ***Clase***: Variable Categorica Ordinal



In [1]:
import pandas as pd

df = pd.DataFrame(data={
    'edad': [20, 30, 40, 50, 60, 70],
    'genero': ['Mas', 'Mas', 'Fem', 'Fem', 'Fem', 'Mas'],
    'ciudad_embarque': ['Cherboarg', 'Queenstown', 'Southampton', 'Cherboarg', 'Queenstown', 'Queenstown'],
    'clase': ['Cuarta', 'Tercera', 'Primera', 'Segunda', 'Segunda', 'Cuarta']
})

df

,edad,genero,ciudad_embarque,clase
0,20,Mas,Cherboarg,Cuarta
1,30,Mas,Queenstown,Tercera
2,40,Fem,Southampton,Primera
3,50,Fem,Cherboarg,Segunda
4,60,Fem,Queenstown,Segunda
5,70,Mas,Queenstown,Cuarta


<hr>


### <a name="M21">2.1. Codificación Binaria</a>


* Vamos a transformar la variable genero a una variable binaria.


* Esto lo podemos hacer con la clase **"LabelBinarizer()"** de scikit-learn de la siguiente manera:

In [2]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
df['genero_binario'] = lb.fit_transform(df['genero'])
df[['genero', 'genero_binario']]

,genero,genero_binario
0,Mas,1
1,Mas,1
2,Fem,0
3,Fem,0
4,Fem,0
5,Mas,1


<hr>


### <a name="M22">2.2. Codificación One Hot Encode</a>


* Vamos a transformar la variable "ciudad_embarque" en tantas nuevas variables como elementos tenga.


* Esto lo podemos hacer con la clase **"OneHotEncoder()"** de scikit-learn de la siguiente manera:

In [3]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
x = df[['ciudad_embarque']].values
x_one_hot = ohe.fit_transform(x).toarray()
x_one_hot

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

* Como vemos nos devuelve una matriz con las codificaciones de cada una de las filas del DataFrame (transformadas a un numpy array).


* Vamos ahora a pasar estas nuevas variables al DataFrame:


In [4]:
df_one_hot = pd.DataFrame(x_one_hot, columns = ["ciudad_embarque_"+str(int(i)) for i in range(x_one_hot.shape[1])])
df_all = pd.concat([df, df_one_hot], axis=1)
df_all

,edad,genero,ciudad_embarque,clase,genero_binario,ciudad_embarque_0,ciudad_embarque_1,ciudad_embarque_2
0,20,Mas,Cherboarg,Cuarta,1,1.0,0.0,0.0
1,30,Mas,Queenstown,Tercera,1,0.0,1.0,0.0
2,40,Fem,Southampton,Primera,0,0.0,0.0,1.0
3,50,Fem,Cherboarg,Segunda,0,1.0,0.0,0.0
4,60,Fem,Queenstown,Segunda,0,0.0,1.0,0.0
5,70,Mas,Queenstown,Cuarta,1,0.0,1.0,0.0


### Codificación One Hot Encode en Pandas


* Esta transformación también nos la permite hacer la librería de Pandas de la siguiente manera:

In [5]:
df_dummy = pd.get_dummies(df['ciudad_embarque'])
df_dummy

,Cherboarg,Queenstown,Southampton
0,1,0,0
1,0,1,0
2,0,0,1
3,1,0,0
4,0,1,0
5,0,1,0


* Concatenamos la codificación resultante al DataFrame

In [6]:
df = pd.concat([df, df_dummy], axis=1)
df

,edad,genero,ciudad_embarque,clase,genero_binario,Cherboarg,Queenstown,Southampton
0,20,Mas,Cherboarg,Cuarta,1,1,0,0
1,30,Mas,Queenstown,Tercera,1,0,1,0
2,40,Fem,Southampton,Primera,0,0,0,1
3,50,Fem,Cherboarg,Segunda,0,1,0,0
4,60,Fem,Queenstown,Segunda,0,0,1,0
5,70,Mas,Queenstown,Cuarta,1,0,1,0


<hr>



### <a name="M23">2.3. Label Encoder</a>


* Vamos a transformar la variable "clase" a una variable numerica.


* Esto lo podemos hacer con la clase **"LabelEncoder()"** de scikit-learn de la siguiente manera:

In [7]:
from sklearn.preprocessing import LabelEncoder

df['clase_endoce'] = LabelEncoder().fit_transform(df['clase'])
df[['clase', 'clase_endoce']]

,clase,clase_endoce
0,Cuarta,0
1,Tercera,3
2,Primera,1
3,Segunda,2
4,Segunda,2
5,Cuarta,0


* Con la clase ***"LabelEncoder()"*** de Scikit tenemos el problema de que nos pone a cada elemento un valor en función del orden alfabetico.


* En el caso de trabajar con variables categóricas ordinales, nos interesa darle un sentido ordinal a los valores de la variable, por lo que vamos a asignarle los valores de la siguiente manera:

In [8]:
df['clase_endoce'] = df['clase'].apply(lambda x: 1 if x == 'Primera' 
                                       else (2 if x == 'Segunda' 
                                             else (3 if x == 'Tercera' else 4)))
df[['clase', 'clase_endoce']]

,clase,clase_endoce
0,Cuarta,4
1,Tercera,3
2,Primera,1
3,Segunda,2
4,Segunda,2
5,Cuarta,4




### Resultado Final:


* Tras realizar todas estas transformaciones ya tendríamos el Dataset transformado correctamente para que pueda ser utilizado por un Algoritmo de Aprendizaje:


In [9]:
df[['edad', 'genero_binario', 'Cherboarg', 'Queenstown', 'Southampton', 'clase_endoce']]

,edad,genero_binario,Cherboarg,Queenstown,Southampton,clase_endoce
0,20,1,1,0,0,4
1,30,1,0,1,0,3
2,40,0,0,0,1,1
3,50,0,1,0,0,2
4,60,0,0,1,0,2
5,70,1,0,1,0,4


<hr>


# <a name="M3">3. Discretización</a>


* ***La discretización o agrupamiento es un proceso en el convierte variables cuantitativas en variables categóricas*** (por lo general categóricas ordinales).


* Este proceso ordena de menor a mayor los valores de la variable y los divide en 'n' rangos proporcionales en número; por ejemplo si queremos discretizar una variable continua en dos categóricas, ordenaremos los valores de menor a mayor y dividiremos los datos a partir de la mediana, asignando como variable categoria 'Q1' a los valores que van del valor mínimo hasta la mediana y con el nombre de 'Q2' a los valores que van desde la mediana hasta el máximo valor.


* Veamos a continuación un ejemplo con la función "qcut" de pandas de como discretizaríamos en 3 rangos la variable edad del Dataframe de ejemplo:

In [10]:
df['quantile_edad_3'] = pd.qcut(df['edad'], 3, labels=['Q1', 'Q2', 'Q3'])
df[['edad', 'quantile_edad_3']].head()

,edad,quantile_edad_3
0,20,Q1
1,30,Q1
2,40,Q2
3,50,Q2
4,60,Q3


* En caso de no asignar una etiqueta a cada grupo, la función nos devuelve el rango de valores al que pertenece cada elemento e información relativa a cada grupo.

In [11]:
pd.qcut(df['edad'], 3)

0    (19.999, 36.667]
1    (19.999, 36.667]
2    (36.667, 53.333]
3    (36.667, 53.333]
4      (53.333, 70.0]
5      (53.333, 70.0]
Name: edad, dtype: category
Categories (3, interval[float64]): [(19.999, 36.667] < (36.667, 53.333] < (53.333, 70.0]]

<hr>


# <a name="M4">4. Normalización</a>


* ***La Normalización es un proceso que consiste en convertir el rango de valores de una variable numérica, a un rango estandar como por ejemplo entre [-1,1], [0,1], etc.***


* Una de las maneras más comunes para normalizar los datos sería la siguiente conocida como el **"Min Max Scaler"**:
<span></span><br><br>
    <span style="font-size:20px">$$\overline{X}^{(i)} = \frac{x^{(i)} - min^{(i)}}{max^{(i)} - min^{(i)}}$$</span>
    
    
* Donde $min^{(j)}$ y $max^{(j)}$ son respectivamente, el valor mínimo y máximo de la variable $j$ en el conjunto de datos.


* Veamos a continuación un ejemplo de normalización, utilizando el siguiente DataFrame:



In [12]:
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

df_origin = pd.DataFrame(data={'X1': [0.0, 1.0,    2.0,    3.0,    4.0,    5.0],
                               'X2': [0.0, 10.0,   20.0,   30.0,   40.0,   50.0],
                               'X3': [0.0, 100.0,  200.0,  300.0,  400.0,  500.0],
                               'X4': [0.0, 1000.0, 2000.0, 3000.0, 4000.0, 5000.0],
                               'y' : [0.0, 4.0,    8.0,    12.0,   16.0,   20.0]
                              })

df_origin

,X1,X2,X3,X4,y
0,0.0,0.0,0.0,0.0,0.0
1,1.0,10.0,100.0,1000.0,4.0
2,2.0,20.0,200.0,2000.0,8.0
3,3.0,30.0,300.0,3000.0,12.0
4,4.0,40.0,400.0,4000.0,16.0
5,5.0,50.0,500.0,5000.0,20.0


* Para normalizar los datos, lo podemos hacer con la clase **"MinMaxScaler()"** de scikit-learn de la siguiente manera, obteniendo el nuevo DataFrame Normalizado.

In [13]:
from sklearn.preprocessing import MinMaxScaler

df_norm = df_origin.copy()

min_max_scaler = MinMaxScaler()

df_norm[['X1','X2','X3','X4']] = min_max_scaler.fit_transform(df_norm[['X1','X2','X3','X4']].values)
df_norm

,X1,X2,X3,X4,y
0,0.0,0.0,0.0,0.0,0.0
1,0.2,0.2,0.2,0.2,4.0
2,0.4,0.4,0.4,0.4,8.0
3,0.6,0.6,0.6,0.6,12.0
4,0.8,0.8,0.8,0.8,16.0
5,1.0,1.0,1.0,1.0,20.0


<hr>


# <a name="M5">5. Estandarización</a>


* La Estandarización es un proceso que consiste en convertir los valores de una variable cuantitativa en valores que hagan seguir todos ellos a la variable en una distribución normal estandar con media '0' y desviación estandar '1'.


* Esto lo conseguimos de la siguiente manera:

<span></span><br><br>
    <span style="font-size:20px">$$\widehat{x}^{(i)} = \frac{x^{(i)} - \mu}{\sigma}$$</span>
    

* Para estandarizar los datos, lo podemos hacer con la clase **"StandardScaler()"** de scikit-learn de la siguiente manera.


In [14]:
from sklearn.preprocessing import StandardScaler

X = [[0], [2], [4]]
scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)
mean = scaler.mean_ 
des = scaler.var_

print("Variable Transformada: \n{}".format(X_scaler))
print("Media de la variable: {}".format(mean))
print("Desviación Estandar de la Variable: {}".format(des))

Variable Transformada: 
[[-1.22474487]
 [ 0.        ]
 [ 1.22474487]]
Media de la variable: [2.]
Desviación Estandar de la Variable: [2.66666667]


<hr>


# <a name="M6">6. Imputación de valores nulos</a>


* Puede ser normal que algunos datos de algunos elementos de nuestro Dataset tengan valores nulos (o valores faltantes) en alguna de las variables y no por ello debamos de descartar ese elemento. Para no descartar dicho elemento debemos de imputarle algún valor a la variable nula del elemento.


* No exite una norma o técnica particular para la imputación de los valores nulos ya que dependerá del objetivo que se quiera conseguir con el Dataset pero si que podemos dar algunas recomendaciones:

    - Si observamos que una variable del Dataset tiene un porcentaje alto de valores nulos, deberíamos de descartar la varible entera.
    - Si la variable es una variable continua podemos:
        + Imputar el valor medio de la variable.
        + Imputar la mediana.
        + Estudiar la distribución de la variable y sacar un número aleatorio siguiendo la distribución de dicha variable.
     - Si la variable es una variable discreta podemos:
        + Imputar el valor más frecuente de la variable.
        + De todos los valores posibles de la variables, asignar uno de esos valores de manera aleatoria.
        + Estudiando la probabilidad de aparición de los valores de la variable, asignar uno de esos valores basandose en la probabilidad de aparición del valor en la variable.

<hr>


Este Notebook ha sido desarrollado por **Ricardo Moya García** y registrado en Safe Creative como ***Atribución-NoComercial-CompartirIgual***.


<img src="../imgs/CC_BY-NC-SA.png" alt="CC BY-NC">